# The following code is also available on the following github: https://github.com/dmunglah/APSMB_coursework_1915222 

In [2]:
# importing the necessary packages 
import sbmltoodepy
import pandas as pd 
import numpy as np 
import scipy
import random
import matplotlib.pyplot as plt
from scipy import linalg, optimize, stats
import scipy.interpolate as interp
import os
import sbmltoodepy
import tkinter
import matplotlib

In [2]:
#Getting the dataset required for the analysis - mouse tumour lung cancer data (Mus musculus)
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE92nnn/GSE92972/suppl/GSE92972_RAW.tar 

--2022-08-06 17:34:36--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE92nnn/GSE92972/suppl/GSE92972_RAW.tar
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.11, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207308800 (198M) [application/x-tar]
Saving to: ‘GSE92972_RAW.tar’

GSE92972_RAW.tar    100%[===================>] 197.71M  39.1MB/s    in 5.6s    

2022-08-06 17:34:42 (35.4 MB/s) - ‘GSE92972_RAW.tar’ saved [207308800/207308800]



In [ ]:
#unzipping the downloaded zipped file and extractiing the data.
!cd university 
!tar -xvf GSE92972_RAW.tar
!gunzip *.gz 
!mkdir dataset_samples_3

In [5]:
!fastqc ./dataset_samples_3/*fastq

Started analysis of ERR015617_1.fastq
Approx 5% complete for ERR015617_1.fastq
Approx 10% complete for ERR015617_1.fastq
Approx 15% complete for ERR015617_1.fastq
Approx 20% complete for ERR015617_1.fastq
Approx 25% complete for ERR015617_1.fastq
Approx 30% complete for ERR015617_1.fastq
Approx 35% complete for ERR015617_1.fastq
Approx 40% complete for ERR015617_1.fastq
Approx 45% complete for ERR015617_1.fastq
Approx 50% complete for ERR015617_1.fastq
Approx 55% complete for ERR015617_1.fastq
Approx 60% complete for ERR015617_1.fastq
Approx 65% complete for ERR015617_1.fastq
Approx 70% complete for ERR015617_1.fastq
Approx 75% complete for ERR015617_1.fastq
Approx 80% complete for ERR015617_1.fastq
Approx 85% complete for ERR015617_1.fastq
Approx 90% complete for ERR015617_1.fastq
Approx 95% complete for ERR015617_1.fastq
Analysis complete for ERR015617_1.fastq
Started analysis of ERR015617_2.fastq
Approx 5% complete for ERR015617_2.fastq
Approx 10% complete for ERR015617_2.fastq
Appr

In [21]:
# trimming with trimmomatic - has been run already
!trimmomatic PE -phred33 ./dataset_samples_3/ERR015617_1.fastq ERR015617_2.fastq ./dataset_samples_3/ERR015618_1.fastq ERR015618_2.fastq  ILLUMINACLIP:contams\_forward\_rev.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36


TrimmomaticPE: Started with arguments:
 -phred33 ./dataset_samples_3/ERR015617_1.fastq ERR015617_2.fastq ./dataset_samples_3/ERR015618_1.fastq ERR015618_2.fastq ILLUMINACLIP:contams_forward_rev.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
Multiple cores found: Using 4 threads
Exception in thread "main" java.io.FileNotFoundException: ERR015617_2.fastq (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at org.usadellab.trimmomatic.fastq.FastqParser.parse(FastqParser.java:135)
	at org.usadellab.trimmomatic.TrimmomaticPE.process(TrimmomaticPE.java:268)
	at org.usadellab.trimmomatic.TrimmomaticPE.run(TrimmomaticPE.java:555)
	at org.usadellab.trimmomatic.Trimmomatic.main(Trimmomatic.java:80)


In [1]:
#Processsing the data using Bash to obtain contigs
!prokka *.fastq --outdir /home/david/university/ --prefix *.fastq
!find . -name /home/david/*.gbk | while read line; do out=$(echo $line | cut -d. -f1-2); python /home/david/tools/gene_to_locus_tool.py $line ${out}.gene_to_contig; done

In [ ]:
#gene_to_locus script
from Bio import SeqIO

import argparse
arg_parser = argparse.ArgumentParser( description = "Parse gbk file and output a gene to locus mapping file" )
arg_parser.add_argument( "gbk_file" )
arg_parser.add_argument( "out_file" )
arguments = arg_parser.parse_args()

gbk_filename = arguments.gbk_file
faa_filename = arguments.out_file

output_handle = open(faa_filename, "w")
for seq_record in SeqIO.parse(gbk_filename, "genbank") :
    print ("Dealing with GenBank record %s" % seq_record.id)
    for seq_feature in seq_record.features :
        if seq_feature.type=="CDS":
            assert len(seq_feature.qualifiers['translation'])==1
            output_handle.write(">%s Start: %s Stop: %s Strand: %s From %s\n" % (
                   seq_feature.qualifiers['locus_tag'][0],
                   seq_feature.location.start,
                   seq_feature.location.end,
                   seq_feature.strand,
                   seq_record.name))

output_handle.close()
print ("Done")

In [ ]:
#Indexing the contigs
!find contigs/*.fa | while read line; do file=$(echo $line | cut -d/ -f2 | cut -d. -f1); mkdir indexed_contigs/${file}; bowtie2-build -f $line indexed_contigs/${file}/${file}; done

In [ ]:
#Creating the bam files needed for analysis
!find contigs/*.fa | while read line; do file=$(echo $line | cut -d/ -f2 | cut -d. -f1); do bowtie2 -p 16 -x indexed_contigs/${file}/${file} -U /datadrive/david/fastq/${file}.fastq.gz | samtools sort > indexed_contigs/${file}/${file}.bam; done

In [ ]:
# Running Kraken and Braken to obtain abundance estimation
!while read line; do identifier=$(echo $line | cut -d/ -f6 | cut -d. -f1); kraken2 --db . ${line} --report kraken_out_3/${identifier}.report --output kraken_out_3/${identifier}.out ; done < sequencing_files.txt
!for i in `cat ids_kraken ` ; do bracken -d /datadrive/david/. -i $i.report -l S -o $i.output.bracken ; done


# Used the following code to combine samples into an abundance table

In [ ]:
#Combine sample files into table using this script

import sys
import glob
import ntpath
from functools import reduce

"""
Input -- 
1.)Directory or Folder containing bracken outputs
2.)Metadata file with no headers, with first column as Bodysite/Status 
    and second column accession number/Sample ID
  * Change the name of columns in the metadata_collist variable according to columns in metadata file 
  * More columns for metadata could be added,by specifying column names in the metadata_collist variable
    
3.) Directory containing kraken outputs
"""

#Function to extract accesion numbers from the bracken output filename
def path_accession(path):
    g = ntpath.basename(path)
    ac = g.split('.')[0]
    return ac

def parse_kraken_report(listofkrakenoutputs):
    listkraken = []
    for kraken in listofkrakenoutputs:
        f = ntpath.basename(kraken)
        filename = f.split('.')
        listkraken.append(pd.read_csv(kraken, sep='\t', header=None, usecols = [1],nrows = 1).assign(name = filename[0]))
    df = pd.concat(listkraken)
    df = df.set_index('name')
    df = df.rename(columns = {1:'Unclassified'})
    return df
        

def main():
  directory = sys.argv[1]
  all_files = glob.glob(directory + "/*.output.bracken")
  metadata_file = sys.argv[2]
  directory_kraken = sys.argv[3]
  kraken_files = glob.glob(directory_kraken + "/*.report")
  kraken_df = parse_kraken_report(kraken_files)
  output_file = 'Bracken_kraken.csv'
  dataframe_list = bracken_to_df(all_files,col_list)
  df_final = merged_df_listofdfs(dataframe_list)
  df_t = df_final.T
  try:
   df_t = df_t.rename(columns=df_t.iloc[0])
   df_t = df_t.drop('name',axis = 0)
  except:
   print('No files with .bracken extension in the current folder')
  accesion_list = [path_accession(path) for path in all_files]
  print(len(accesion_list))
  print(df_t.shape)
  df_t.index = accesion_list
  #metadata = pd.DataFrame()
  #try:
  #metadata = pd.read_csv(metadata_file,header = None,names = metadata_collist,sep = '\t', engine= 'c')
  #metadata = metadata.set_index('sample_id')
  #except:
  #print('Metadata file does not have column names specified on metadata_collist variable')
  final_list = [df_t,kraken_df]
  tax_matrix = reduce(lambda left,right: pd.merge(left,right,left_index = True, right_index = True),final_list) 
  ##tax_matrix =  pd.merge(df_t,metadata,left_index = True,right_index= True)
  print(tax_matrix)
  tax_matrix.to_csv(output_file,sep = '\t')
 
if __name__ == "__main__":
    main()

In [4]:
# Reading in the data and utilising an N-dimetional arrray to process the data. 
filedata_1 = pd.read_csv('/datadrive/david/Bracken_kraken11.tsv', sep='\t')
filedata_1.head
filedata_2d = np.array(filedata_1)
filedata_2d.shape
#arr_2d = np.reshape(filedata, (3, 1))
#arr_2d.shape

(37, 2496)

In [11]:
# downloading a a suitable model
!wget https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000427.2?filename=BIOMD0000000427_url.xml

--2022-08-08 11:04:59--  https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000427.2?filename=BIOMD0000000427_url.xml
Resolving www.ebi.ac.uk (www.ebi.ac.uk)... 193.62.193.80
Connecting to www.ebi.ac.uk (www.ebi.ac.uk)|193.62.193.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/xml]
Saving to: ‘BIOMD0000000427.2?filename=BIOMD0000000427_url.xml.1’

BIOMD0000000427.2?f     [ <=>                ]  53.63K  --.-KB/s    in 0.09s   

2022-08-08 11:04:59 (568 KB/s) - ‘BIOMD0000000427.2?filename=BIOMD0000000427_url.xml.1’ saved [54914]



In [12]:
#intalling sbmltoodepy
!pip install sbmltoodepy

In [13]:
# Identified a model that contains a representation of a part of my system 
sbmltoodepy.ParseAndCreateModel("/home/david/university/BIOMD0000000427.2?filename=BIOMD0000000427_url.xml", outputFilePath = "Lockefile.py", className = "Lockemodel")
from Lockefile import Lockemodel
model = Lockemodel()

In [14]:
# Get the dictionary keys for the IDs of the parameters in the model
print(model.p.keys())
print(model.s.keys())
print(model.c.keys())

dict_keys(['gamma_IGFR', 'kd_PI3K_a', 'k_P90Rsk_ERKActive', 'KM_P90Rsk_ERKActive', 'gamma_EGFR'])
dict_keys(['EGFR_active', 'D_SOS', 'A_SOS', 'Raf', 'Ras_active', 'Mek_active', 'ERK', 'ERK_active', 'IGFR_active', 'PI3KCA', 'PI3KCA_active', 'AKT_active', 'AKT', 'PP2A', 'Ras', 'Raf_active', 'Mek', 'RasGapActive', 'RafPP', 'P90RskInactive', 'P90Rsk_Active'])
dict_keys(['cell_nsclc'])


In [15]:
# replace compartmentId with one of the dictionary keys returned from print(modelInstance.c.keys())
print(model.c['cell_nsclc'].size) 
# replace parameterId with one of the dictionary keys returned from print(modelInstance.p.keys())
print(model.p['kd_PI3K_a'].value)
# replace speciesId with one of the dictionary keys returned from print(modelInstance.s.keys())
print(model.s['EGFR_active'].concentration)
print(model.s['EGFR_active'].amount)

1.0
0.005
8000.0
8000.0


In [16]:
print(model.time)

0


In [39]:
#solving differential equation
# function that returns dy/dt
from scipy.integrate import odeint
y = odeint(model, y0, t)

def model(y,t):
    dydt = -y + 1.0
    return dydt

# initial condition
y0 = 0

# time points
t = np.linspace(0,0.005)

# solve ODE
y = odeint(model,y0,t)

# plot results
plt.plot(t,y)
plt.xlabel('time')
plt.ylabel('y(t)')
plt.show()

In [17]:
timeinterval = 1
model.RunSimulation(timeinterval)

In [18]:
print(model.time)

1


In [19]:
times = np.zeros(101)
times[0] = model.time
concentrations = np.zeros(101)
concentrations[0] = model.s['EGFR_active'].concentration
timeinterval = 1
for i in range(100):
    model.RunSimulation(timeinterval)
    times[i+1] = model.time
    concentrations[i+1] = model.s['EGFR_active'].concentration

In [20]:
#Plotting 
import tkinter
matplotlib.use('TkAgg')
plt.plot(times,concentrations)
#plt.show() #displays figure via X11 - will be on git hub page

In [32]:
# Attempting to plot network
import networkx as nx

adjacency_matrix = filedata_2d
def squarify(adjacency_matrix,val):
    (a,b)=adjacency_matrix.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return numpy.pad(adjacency_matrix,padding,mode='constant',constant_values=val)
#print(adjacency_matrix)

In [35]:
#G = nx.Graph(adjacency_matrix, nodetype=int)
#G.edges()
G = nx.Graph()
G.add_edges_from('/home/david/abundance_tables/sample_list.txt)
nx.draw_networkx(G)